# Demo Notebook For Training and Running Zingg Entity Resolution Workflow on Databricks
This notebook runs the Zingg Febrl Example on Databricks. 
Please refer to the 
1. [Zingg Python API](https://readthedocs.org/projects/zingg/) 
2. [Zingg Official Documentation](https://www.docs.zingg.ai) 
for details.

Please ensure your cluster has the following installed
1. Zingg from pypi
2. Zingg jar from the [repo](https://github.com/zinggAI/zingg/releases)
3. tabulate from pypi

Please execute each cell one by one as per the instructions provided.

If you face any issue, please [log an issue](https://github.com/zinggAI/zingg/issues)

You can also join [Zingg's Slack community](https://join.slack.com/t/zinggai/shared_invite/zt-w7zlcnol-vEuqU9m~Q56kLLUVxRgpOA)

In [0]:
pip install zingg

Python interpreter will be restarted.
Python interpreter will be restarted.


In [0]:
pip show zingg

Name: zingg
Version: 0.4.0
Summary: Zingg Entity Resolution, Data Mastering and Deduplication
Home-page: https://github.com/zinggAI/zingg
Author: Zingg.AI
Author-email: sonalgoyal4@gmail.com
License: https://github.com/zinggAI/zingg/blob/main/LICENSE
Location: /local_disk0/.ephemeral_nfs/cluster_libraries/python/lib/python3.9/site-packages
Requires: py4j
Required-by: 


In [0]:
pip install tabulate

Python interpreter will be restarted.
Python interpreter will be restarted.


In [0]:
pip show tabulate

Name: tabulate
Version: 0.8.9
Summary: Pretty-print tabular data
Home-page: https://github.com/astanin/python-tabulate
Author: Sergey Astanin
Author-email: s.astanin@gmail.com
License: MIT
Location: /databricks/python3/lib/python3.9/site-packages
Requires: 
Required-by: databricks-cli


# Define locations for the model
The Zingg models and training data are persisted in dbfs. 

**Please edit the model id in the cell below to reflect your model.**

In [0]:
##you can change these to the locations of your choice
##these are the only two settings that need to change
zinggDir = "/models"
modelId = "databricksdemotrial"

# Setup common functions for use in Zingg. 
These functions setup the internal folders used by Zingg, and help with labeling and training Zingg. 

**No change is needed in the cell below.**

In [0]:

##please leave the following unchanged
MARKED_DIR = zinggDir + "/" + modelId + "/trainingData/marked/"
UNMARKED_DIR = zinggDir + "/" + modelId + "/trainingData/unmarked/"

MARKED_DIR_DBFS = "/dbfs" + MARKED_DIR
UNMARKED_DIR_DBFS = "/dbfs" + UNMARKED_DIR  


import pandas as pd
import numpy as np
 
import time
import uuid
 
from tabulate import tabulate
from ipywidgets import widgets, interact, GridspecLayout
import base64

import pyspark.sql.functions as fn

##this code sets up the Zingg Python interface
from zingg.client import *
from zingg.pipes import *

def cleanModel():
    dbutils.fs.rm(MARKED_DIR, recurse=True)
    # drop unmarked data
    dbutils.fs.rm(UNMARKED_DIR, recurse=True)
    return

# assign label to candidate pair
def assign_label(candidate_pairs_pd, z_cluster, label):
  '''
  The purpose of this function is to assign a label to a candidate pair
  identified by its z_cluster value.  Valid labels include:
     0 - not matched
     1 - matched
     2 - uncertain
  '''
  
  # assign label
  candidate_pairs_pd.loc[ candidate_pairs_pd['z_cluster']==z_cluster, 'z_isMatch'] = label
  
  return
 
def count_labeled_pairs(marked_pd):
  '''
  The purpose of this function is to count the labeled pairs in the marked folder.
  '''

  n_total = len(np.unique(marked_pd['z_cluster']))
  n_positive = len(np.unique(marked_pd[marked_pd['z_isMatch']==1]['z_cluster']))
  n_negative = len(np.unique(marked_pd[marked_pd['z_isMatch']==0]['z_cluster']))
  
  return n_positive, n_negative, n_total

# setup widget 
available_labels = {
    'No Match':0,
    'Match':1,
    'Uncertain':2
    }
#dbutils.widgets.dropdown('label', 'Uncertain', available_labels.keys(), 'Is this pair a match?')




/databricks/spark/python/pyspark/sql/context.py:117: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  warnings.warn(


# Start building the Zingg program 
The following cell sets up the initial arguments for Zingg. 

**No change is needed in the cell below.**

In [0]:

#build the arguments for zingg
args = Arguments()
# Set the modelid and the zingg dir. You can use this as is
args.setModelId(modelId)
args.setZinggDir(zinggDir)


# Define the input
Our data is in csv so we provide a schema. You can choose other formats like parquet by using Pipe with parquet as the format.
You can also pass in a dataframe by using a Pipe with the in memory format. 
Please refer to [Pipes](https://zingg.readthedocs.io/en/latest/zingg.html#zingg.pipes.Pipe) for details on different formats

**Please modify this for your data.**

In [0]:
schema = "id string, fname string, lname string, stNo string, add1 string, add2 string, city string, state string, areacode string, dob string, ssn  string"
inputPipe = CsvPipe("testFebrl", "/FileStore/tables/test.csv", schema)

args.setData(inputPipe)

set schema 


# Configure the output
Here we configure the putput to be a csv, but similar to the input above, the output can be a file format like parquet or delta or a data store like MySQL

**Please modify this for your data.**

In [0]:
#setting outputpipe in 'args'
outputPipe = CsvPipe("resultFebrl", "/tmp/febrlOutput")
args.setOutput(outputPipe)

# Define the match fields and their types

The cell below is used to configure Zingg with the fields for use in matching and the match types.
Details on the field definitions can be found at [Zingg official docs](https://www.docs.zingg.ai)

**Please modify this for your data.**

In [0]:
#set field definitions 
#please change these 
fname = FieldDefinition("fname", "string", MatchType.FUZZY)
lname = FieldDefinition("lname", "string", MatchType.FUZZY)
stNo = FieldDefinition("stNo", "string", MatchType.FUZZY)
add1 = FieldDefinition("add1","string", MatchType.FUZZY)
add2 = FieldDefinition("add2", "string", MatchType.FUZZY)
city = FieldDefinition("city", "string", MatchType.FUZZY)
areacode = FieldDefinition("areacode", "string", MatchType.FUZZY)
state = FieldDefinition("state", "string", MatchType.FUZZY)
dob = FieldDefinition("dob", "string", MatchType.FUZZY)
ssn = FieldDefinition("ssn", "string", MatchType.FUZZY)

fieldDefs = [fname, lname, stNo, add1, add2, city, areacode, state, dob, ssn]
args.setFieldDefinition(fieldDefs)



# Performance settings

The numPartitions define how data is split across the cluster. Please change this as per your data and cluster size by referring to the performance section of the Zingg docs.
The labelDataSampleSize is used for sampling in findTrainingData. It lets Zingg select pairs for labeling in a reasonable amount of time. 
If the findTrainingData phase is taking to much time, please reduce this by atleast 1/10th of its previous value and try again.

**Please modify this for your data.**

In [0]:

# The numPartitions define how data is split across the cluster. 
# Please change the fllowing as per your data and cluster size by referring to the docs.

args.setNumPartitions(4)
args.setLabelDataSampleSize(0.5)



# Get Zingg to find pairs for user labeling
**No change is needed in the cell below.**

In [0]:
options = ClientOptions([ClientOptions.PHASE,"findTrainingData"])

#Zingg execution for the given phase
zingg = ZinggWithSpark(args, options)
zingg.initAndExecute()

['--phase', 'findTrainingData']
arguments for client options are  ['--phase', 'findTrainingData', '--license', 'zinggLic.txt', '--email', 'zingg@zingg.ai', '--conf', 'dummyConf.json']


# Prepare for user labeling
**No change is needed in the cell below.**

In [0]:
options = ClientOptions([ClientOptions.PHASE,"label"])

#Zingg execution for the given phase
zingg = ZinggWithSpark(args, options)
zingg.init()

['--phase', 'label']
arguments for client options are  ['--phase', 'label', '--license', 'zinggLic.txt', '--email', 'zingg@zingg.ai', '--conf', 'dummyConf.json']


# See if we have records for labeling

**No change is needed to the cell below.**

In [0]:
# get candidate pairs
candidate_pairs_pd = getPandasDfFromDs(zingg.getUnmarkedRecords())
 
# if no candidate pairs, run job and wait
if candidate_pairs_pd.shape[0] == 0:
  print('No unlabeled candidate pairs found.  Run findTraining job ...')

else:
    # get list of pairs (as identified by z_cluster) to label 
    z_clusters = list(np.unique(candidate_pairs_pd['z_cluster'])) 

    # identify last reviewed cluster
    last_z_cluster = '' # none yet

    # print candidate pair stats
    print('{0} candidate pairs found for labeling'.format(len(z_clusters)))
  

/databricks/spark/python/pyspark/sql/dataframe.py:149: UserWarning: DataFrame constructor is internal. Do not directly use it.
  warnings.warn("DataFrame constructor is internal. Do not directly use it.")


20 candidate pairs found for labeling


# Start labeling to make Zingg learn how we want to match our data

**No change is needed to the cell below.**

In [0]:
# Label Training Set

# define variable to avoid duplicate saves
ready_for_save = False
print(candidate_pairs_pd)

# user-friendly labels and corresponding zingg numerical value
# (the order in the dictionary affects how displayed below)
LABELS = {
  'Uncertain':2,
  'Match':1,
  'No Match':0  
  }

# GET CANDIDATE PAIRS
# ========================================================
#candidate_pairs_pd = get_candidate_pairs()
n_pairs = int(candidate_pairs_pd.shape[0]/2)
# ========================================================

# DEFINE IPYWIDGET DISPLAY
# ========================================================
display_pd = candidate_pairs_pd.drop(
  labels=[
    'z_zid', 'z_prediction', 'z_score', 'z_isMatch', 'z_zsource'
    ], 
  axis=1)

# define header to be used with each displayed pair
html_prefix = "<p><span style='font-family:Courier New,Courier,monospace'>"
html_suffix = "</p></span>"
header = widgets.HTML(value=f"{html_prefix}<b>" + "<br />".join([str(i)+"&nbsp;&nbsp;" for i in display_pd.columns.to_list()]) + f"</b>{html_suffix}")

# initialize display
vContainers = []
vContainers.append(widgets.HTML(value=f'<h2>Indicate if each of the {n_pairs} record pairs is a match or not</h2></p>'))

# for each set of pairs
for n in range(n_pairs):

  # get candidate records
  candidate_left = display_pd.loc[2*n].to_list()
  print(candidate_left)
  candidate_right = display_pd.loc[(2*n)+1].to_list()
  print(candidate_right)

  # define grid to hold values
  html = ''

  for i in range(display_pd.shape[1]):

    # get column name
    column_name = display_pd.columns[i]

    # if field is image
    if column_name == 'image_path':

      # define row header
      html += '<tr>'
      html += '<td><b>image</b></td>'

      # read left image to encoded string
      l_endcode = ''
      if candidate_left[i] != '':
        with open(candidate_left[i], "rb") as l_file:
          l_encode = base64.b64encode( l_file.read() ).decode()

      # read right image to encoded string
      r_encode = ''
      if candidate_right[i] != '':
        with open(candidate_right[i], "rb") as r_file:
          r_encode = base64.b64encode( r_file.read() ).decode()      

      # present images
      html += f'<td><img src="data:image/png;base64,{l_encode}"></td>'
      html += f'<td><img src="data:image/png;base64,{r_encode}"></td>'
      html += '</tr>'

    elif column_name != 'image_path':  # display text values

      if column_name == 'z_cluster': z_cluster = candidate_left[i]

      html += '<tr>'
      html += f'<td style="width:10%"><b>{column_name}</b></td>'
      html += f'<td style="width:45%">{str(candidate_left[i])}</td>'
      html += f'<td style="width:45%">{str(candidate_right[i])}</td>'
      html += '</tr>'

  # insert data table
  table = widgets.HTML(value=f'<table data-title="{z_cluster}" style="width:100%;border-collapse:collapse" border="1">'+html+'</table>')
  z_cluster = None

  # assign label options to pair
  label = widgets.ToggleButtons(
    options=LABELS.keys(), 
    button_style='info'
    )

  # define blank line between displayed pair and next
  blankLine=widgets.HTML(value='<br>')

  # append pair, label and blank line to widget structure
  vContainers.append(widgets.VBox(children=[table, label, blankLine]))

# present widget
display(widgets.VBox(children=vContainers))
# ========================================================

# mark flag to allow save 
ready_for_save = True


    z_zid         z_cluster  z_prediction  ...       dob      ssn  z_zsource
0      37   1701239388135:0           1.0  ...  19280722  7626396  testFebrl
1      36   1701239388135:0           1.0  ...  19280722  7626396  testFebrl
2      30   1701239388135:1           0.0  ...  19191031  7801066  testFebrl
3      18   1701239388135:1           0.0  ...  19461101  4783085  testFebrl
4      16  1701239388135:10           1.0  ...  19180205  1909717  testFebrl
5      15  1701239388135:10           1.0  ...  19180205  9341716  testFebrl
6      59  1701239388135:11           0.0  ...  19670206  9724789  testFebrl
7       8  1701239388135:11           0.0  ...  19410111  2540080  testFebrl
8      47  1701239388135:12           1.0  ...  19250817  2655081  testFebrl
9      46  1701239388135:12           1.0  ...  19250817  2655081  testFebrl
10     49  1701239388135:13           0.0  ...  19050820  3677127  testFebrl
11      0  1701239388135:13           0.0  ...  19271027  2402765  testFebrl

# Save all the labels provided by the user 
No change is needed to the cell below.

In [0]:
if not ready_for_save:
  print('No labels have been assigned. Run the previous cell to create candidate pairs and assign labels to them before re-running this cell.')

else:

  # ASSIGN LABEL VALUE TO CANDIDATE PAIRS IN DATAFRAME
  # ========================================================
  # for each pair in displayed widget
  for pair in vContainers[1:]:

    # get pair and assigned label
    html_content = pair.children[1].get_interact_value() # the displayed pair as html
    user_assigned_label = pair.children[1].get_interact_value() # the assigned label

    # extract candidate pair id from html pair content
    start = pair.children[0].value.find('data-title="')
    if start > 0: 
      start += len('data-title="') 
      end = pair.children[0].value.find('"', start+2)
    pair_id = pair.children[0].value[start:end]



    # assign label to candidate pair entry in dataframe
    candidate_pairs_pd.loc[candidate_pairs_pd['z_cluster']==pair_id, 'z_isMatch'] = LABELS.get(user_assigned_label)
  # ========================================================

  # SAVE LABELED DATA TO ZINGG FOLDER
  # ========================================================
  # make target directory if needed
  dbutils.fs.mkdirs(MARKED_DIR)

  # save label assignments
  # save labels
  zingg.writeLabelledOutputFromPandas(candidate_pairs_pd,args)

  # count labels accumulated
  marked_pd_df = getPandasDfFromDs(zingg.getMarkedRecords())
  n_pos, n_neg, n_tot = count_labeled_pairs(marked_pd_df)
  print(f'You have accumulated {n_pos} pairs labeled as positive matches.')
  print(f'You have accumulated {n_neg} pairs labeled as not matches.')
  print("If you need more pairs to label, re-run the cell for 'findTrainingData'")
  # ========================================================  

  # save completed
  ready_for_save = False

You have accumulated 12 pairs labeled as positive matches.
You have accumulated 12 pairs labeled as not matches.
If you need more pairs to label, re-run the cell for 'findTrainingData'


/databricks/spark/python/pyspark/sql/dataframe.py:149: UserWarning: DataFrame constructor is internal. Do not directly use it.
  warnings.warn("DataFrame constructor is internal. Do not directly use it.")


# Persist the Zingg models by training on the labels and run them to predict matches
No change is needed to the cell below.

In [0]:
options = ClientOptions([ClientOptions.PHASE,"trainMatch"])

#Zingg execution for the given phase
zingg = ZinggWithSpark(args, options)
zingg.initAndExecute()

['--phase', 'trainMatch']
arguments for client options are  ['--phase', 'trainMatch', '--license', 'zinggLic.txt', '--email', 'zingg@zingg.ai', '--conf', 'dummyConf.json']


# Voila! We are done. Lets check the results. 

**No change is needed to the cell below.**

In [0]:
outputDF = spark.read.csv('/tmp/febrlOutput')

In [0]:

colNames = ["z_minScore", "z_maxScore", "z_cluster","fname", "lname", "stNo", "add1", "add2", "city", "areacode", "state", "dob", "ssn"]
outputDF.toDF(*colNames).show(100)

+------------------+------------------+---------+---------+-------------+----+-------------------+--------------------+----------------+--------+-----+--------+-------+
|        z_minScore|        z_maxScore|z_cluster|    fname|        lname|stNo|               add1|                add2|            city|areacode|state|     dob|    ssn|
+------------------+------------------+---------+---------+-------------+----+-------------------+--------------------+----------------+--------+-----+--------+-------+
|0.9999999999186376|0.9999999999186376|        1|   thomas|       george|   1|      mcmanus place|        stoney creek|north turramurra|      sa| 3130|19630225|5460534|
|0.9999877574360159|0.9999877574360159|        3|     Érik|         Guay| 840|      fowles street|         moun tvjiew|  burleigh heads|      ss| 2830|    null|2932837|
|0.9999997560138284|0.9999998455488001|        5|  jackson|      christo| 840|      fowles street|          mou ntveiw|   burleig heads|      sa| 2830|1983